In [0]:
sales_df = spark.table("config_catalog.silver.sales")

dim_customers = spark.table("config_catalog.gold.dim_customers").filter("is_current = true")
dim_products  = spark.table("config_catalog.gold.dim_products").filter("is_current = true")

fact_df = sales_df \
    .join(dim_customers, "customer_id") \
    .join(dim_products, "product_id") \
    .select(
        "order_id",
        "customer_sk",
        "product_sk",
        "amount",
        "order_date"
    )

fact_df.write.format("delta").mode("append").saveAsTable("config_catalog.gold.fact_sales")

In [0]:
dbutils.widgets.text("run_id", "")
run_id = dbutils.widgets.get("run_id")

dbutils.widgets.text("pipeline_name", "")
pipeline_name = dbutils.widgets.get("pipeline_name")



jdbc_url = "jdbc:sqlserver://configserver18.database.windows.net:1433;database=config_db"

connection_props = {
    "user": 'Mahi_123',
    "password":'Maram_098',
    "driver": "com.microsoft.sqlserver.jdbc.SQLServerDriver"
}
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, TimestampType

from pyspark.sql.types import *
from pyspark.sql import Row
from datetime import datetime
from pyspark.sql.functions import col

schema = StructType([
    StructField("pipeline_name", StringType(), True),
    StructField("run_id", StringType(), True),
    StructField("dataset_name", StringType(), True),
    StructField("status", StringType(), True),
    StructField("rows_copied", IntegerType(), True),
    StructField("error_message", StringType(), True),
    StructField("end_time", TimestampType(), True)
])



success_row = [Row(
        pipeline_name=pipeline_name,
        run_id=run_id,
        dataset_name="FACT_SALES",
        status="SUCCESS",
        rows_copied=int(rows_copied),
        error_message=None,
        end_time=datetime.now()
    )]

    success_df = spark.createDataFrame(success_row, schema)
    success_df.write.jdbc(url=jdbc_url, table="pipeline_log", mode="append", properties=connection_props)